In [1]:
import zipfile
from pathlib import Path

zip_path = Path("data/Preprocesed Train Data Side camera.zip")
extract_path = Path("data/custom_data")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [2]:
!python train_val_split.py --datapath="data/custom_data/Preprocesed Data" --train_pct=0.8

Number of image files: 1350
Number of annotation files: 1350
Images moving to train: 1080
Images moving to validation: 270


In [3]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': 'data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = 'data/custom_data/Preprocesed Data/classes.txt'
path_to_data_yaml = 'data/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')

file_path = Path("data/data.yaml")

with open(file_path, "r", encoding="utf-8") as f:
    print(f.read())

Created config file at data/data.yaml

File contents:

path: data
train: train/images
val: validation/images
nc: 11
names:
- 01_Good
- 10_Overhang_Sag
- 11_Shifted_Layer
- 12_Warping
- 13_Delamination
- 14_Unclear
- 15_Not_Printing
- 4_Foreign_Body
- 6_Stringing
- 7_Spaghetti
- 9_Poor_Bridging



In [5]:
yaml_text = """path: data
train: train/images
val: validation/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing
"""

Path("data/data.yaml").write_text(yaml_text, encoding="utf-8")
print(Path("data/data.yaml").read_text(encoding="utf-8"))

path: data
train: train/images
val: validation/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing



In [6]:
from pathlib import Path
from collections import Counter

root = Path("data")
label_files = list(root.rglob("labels/*.txt")) + list(root.rglob("**/labels/**/*.txt"))
print("Found label files:", len(label_files))

counts = Counter()
max_id = -1

for f in label_files:
    for line in f.read_text(encoding="utf-8", errors="ignore").splitlines():
        if not line.strip():
            continue
        cls = int(float(line.split()[0]))
        counts[cls] += 1
        max_id = max(max_id, cls)

print("Class IDs present:", sorted(counts.keys()))
print("Max class ID:", max_id)
print("Counts:", dict(sorted(counts.items())))

Found label files: 6272
Class IDs present: [1, 2, 3, 4, 5, 6, 7, 8]
Max class ID: 8
Counts: {1: 2070, 2: 1118, 3: 1214, 4: 836, 5: 2492, 6: 1064, 7: 1326, 8: 3126}


In [7]:
import sys, torch
import ultralytics

print("Python exe:", sys.executable)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("Ultralytics version:", ultralytics.__version__)

Python exe: c:\Program Files\Python312\python.exe
Torch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 4060 Laptop GPU
Ultralytics version: 8.3.229


In [2]:
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt" 
DEVICE = 0                        
PROJECT = "runs/gridsearch"       
GRID_EPOCHS = 10
FIXED_IMGSZ = 640                        
SEED = 42

Hyperparameter tuning with Grid-Search

In [8]:
from ultralytics import YOLO
from pathlib import Path
import itertools
import pandas as pd
import time

# CONFIG
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt"          
DEVICE = 0                        
PROJECT = "runs/gridsearch"       
GRID_EPOCHS = 5
FIXED_IMGSZ = 640                        
SEED = 42

# GRID SETUP
grid = {
    "lr0": [0.01, 0.001],
    "weight_decay": [0.0005, 0.005, 0.05],
    "box": [7.5, 10],
    "freeze": [[i for i in range(22)], [i for i in range(21)], [i for i in range(20)]]
}

Path(PROJECT).mkdir(parents=True, exist_ok=True)

keys = list(grid.keys())
combos = list(itertools.product(*[grid[k] for k in keys]))
print(f"Total runs: {len(combos)}")

rows = []
t0 = time.time()

for i, values in enumerate(combos, start=1):
    params = dict(zip(keys, values))
    run_name = "gs_" + "_".join([f"{k}={v}" for k, v in params.items()])

    print(f"\n[{i}/{len(combos)}] Training: {run_name}")
    model = YOLO(BASE_MODEL)

    train_results = model.train(
        data=DATA_YAML,
        epochs=GRID_EPOCHS,
        imgsz=FIXED_IMGSZ,
        lr0=params["lr0"],
        weight_decay=params["weight_decay"],
        optimizer="AdamW",
        box=params["box"],
        freeze=params["freeze"],
        batch=16,
        device=DEVICE,
        seed=SEED,
        deterministic=True,
        project=PROJECT,
        name=run_name,
        plots=False,           # faster
        save=True,
        val=True,
        patience=4            # early stopping helps speed grid search
    )

# After training, evaluate the best checkpoint on val to get a clean final score
    best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
    best_model = YOLO(str(best_path))
    val_metrics = best_model.val(data=DATA_YAML, split="val", device=DEVICE, plots=False)

    # Extract metrics (these attribute names are stable in Ultralytics v8.x)
    row = {
        "run_name": run_name,
        "save_dir": str(model.trainer.save_dir),
        "best_pt": str(best_path),
        **params,
        "imgsz": params.get("imgsz", FIXED_IMGSZ),
        "val_mAP50_95": float(val_metrics.box.map),     # mAP@[.5:.95]
        "val_mAP50": float(val_metrics.box.map50),      # mAP@.5
        "val_precision": float(val_metrics.box.mp),     # mean precision
        "val_recall": float(val_metrics.box.mr),        # mean recall
    }
    rows.append(row)

 # Write partial results continuously (so you don’t lose progress)
    df = pd.DataFrame(rows).sort_values("val_mAP50_95", ascending=False)
    df.to_csv(Path(PROJECT) / "grid_results.csv", index=False)

print(f"\nDone in {(time.time()-t0)/60:.1f} minutes.")
df.head(10)

Total runs: 36

[1/36] Training: gs_lr0=0.01_weight_decay=0.0005_box=7.5_freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_de

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000023136C10E00>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000023136C10E00>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\d

optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\runs\gridsearch\gs_lr0=0.01_weight_decay=0.0005_box=7.5_freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        1/5     0.893G      2.579      3.639      1.991         16        640: 100% ━━━━━━━━━━━━ 82/82 7.2it/s 11.3s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 6.2it/s 2.6s0.2s
                   all        488       1030      0.269      0.312      0.214     0.0744

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        2/5      1.03G      2.327      2.00

,run_name,save_dir,best_pt,lr0,weight_decay,box,freeze,imgsz,val_mAP50_95,val_mAP50,val_precision,val_recall
32,gs_lr0=0.001_weight_decay=0.05_box=7.5_freeze=...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0500,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.287061,0.581648,0.600482,0.542962
31,gs_lr0=0.001_weight_decay=0.05_box=7.5_freeze=...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0500,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.287061,0.581648,0.600482,0.542962
30,gs_lr0=0.001_weight_decay=0.05_box=7.5_freeze=...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0500,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.286351,0.583664,0.596918,0.546434
24,gs_lr0=0.001_weight_decay=0.005_box=7.5_freeze...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0050,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.285224,0.578175,0.592247,0.565711
19,gs_lr0=0.001_weight_decay=0.0005_box=7.5_freez...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.283981,0.579389,0.622795,0.537720
20,gs_lr0=0.001_weight_decay=0.0005_box=7.5_freez...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.283981,0.579389,0.622795,0.537720
21,gs_lr0=0.001_weight_decay=0.0005_box=10_freeze...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,10.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.281833,0.575204,0.617637,0.505923
18,gs_lr0=0.001_weight_decay=0.0005_box=7.5_freez...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,7.5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.281807,0.581471,0.610700,0.544940
23,gs_lr0=0.001_weight_decay=0.0005_box=10_freeze...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,10.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.281804,0.580917,0.607133,0.532473
22,gs_lr0=0.001_weight_decay=0.0005_box=10_freeze...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,10.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",640,0.281804,0.580917,0.607133,0.532473


In [ ]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path

PROJECT = "runs/gridsearch"

df = pd.read_csv(Path(PROJECT) / "grid_results.csv").sort_values("val_mAP50_95", ascending=False)
best = df.iloc[0].to_dict()
print("Best config:", best)

final_name = "best_hyperparameters"
model = YOLO(BASE_MODEL)
model.train(data="data/data.yaml",
            epochs=150,
            imgsz=FIXED_IMGSZ,
            lr0=float(best["lr0"]),
            weight_decay=float(best["weight_decay"]),
            optimizer=str(best["optimizer"]),
            box=float(best["box"]),
            freeze=best["freeze"],
            batch=16,
            device=DEVICE,
            seed=SEED,
            deterministic=True,
            project="runs/detect",
            name=final_name,
            plots=True,
            val=True,
            patience=10
)

best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
last_path = Path(model.trainer.save_dir) / "weights" / "last.pt"
print("✅ Best checkpoint:", best_path)
print("✅ Last checkpoint:", last_path)

Best config: {'run_name': 'gs_lr0=0.001_weight_decay=0.05_optimizer=AdamW_box=7.5', 'save_dir': 'C:\\Users\\User\\Documents\\GitHub\\Real-Time-Defect-Detection-Capstone-Project\\runs\\gridsearch\\gs_lr0=0.001_weight_decay=0.05_optimizer=AdamW_box=7.5', 'best_pt': 'C:\\Users\\User\\Documents\\GitHub\\Real-Time-Defect-Detection-Capstone-Project\\runs\\gridsearch\\gs_lr0=0.001_weight_decay=0.05_optimizer=AdamW_box=7.5\\weights\\best.pt', 'lr0': 0.001, 'weight_decay': 0.05, 'optimizer': 'AdamW', 'box': 7.5, 'imgsz': 640, 'val_mAP50_95': 0.4983985592374256, 'val_mAP50': 0.864041683641595, 'val_precision': 0.8291439996393639, 'val_recall': 0.7885616737639672}
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_m

KeyboardInterrupt: 

In [6]:
model = YOLO("runs/detect/best_hyperparameters/weights/best.pt")

In [14]:
model.predict(source="data/validation/images",
              save=True,
              name="val"
)

print("✅ Saved validation predictions under runs/detect/predict*")


image 1/488 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\00973b46-right_photo_90_2024-10-22_17-18-51.jpg: 416x640 3 13_Delaminations, 59.7ms
image 2/488 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\00a1e045-233photo_2024-08-21T16_37_51-191689.jpg: 416x640 2 12_Warpings, 57.7ms
image 3/488 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\00b9f800-left_163photo_2024-10-02T14_05_41-353505.jpg: 416x640 1 6_Stringing, 63.4ms
image 4/488 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\012f441e-left_photo_57_2024-10-22_16-54-09.jpg: 416x640 1 13_Delamination, 42.4ms
image 5/488 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\01d84a5c-328photo_2024-09-30T15_28_00-283539.jpg: 416x640 1 9_Poor_Bridging, 45.2ms
image 6/488 c:\Users\User\Document

In [8]:
from ultralytics import YOLO

model = YOLO("runs/detect/best_hyperparameters/weights/best.pt")

In [11]:
model.predict(
    source="data/test/images",  # wherever your test images are
    save=True,
    name="test",
    save_txt=True,     # saves YOLO-format predictions (class x y w h conf) per image
    save_conf=True
)


image 1/200 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\00624767-right_77photo_2024-10-04T11_08_33-787900.jpg: 416x640 1 4_Foreign_Body, 1 7_Spaghetti, 59.3ms
image 2/200 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\00a8a190-left_1750photo_2024-08-27T17_19_02-958622.jpg: 416x640 4 4_Foreign_Bodys, 63.3ms
image 3/200 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\042994e7-left_right_181photo_2024-10-02T14_43_58-554388.jpg: 416x640 (no detections), 47.1ms
image 4/200 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\04fee4c2-left_photo_232_2024-10-07_15-43-52.jpg: 416x640 (no detections), 4.6ms
image 5/200 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\images\0654987c-left_210photo_2024-10-02T14_09_45-691906.jpg: 416x640 2 6_Stringings, 6.5ms
image 6/200 c:\Users\User\Do

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '01_Good', 1: '4_Foreign_Body', 2: '6_Stringing', 3: '7_Spaghetti', 4: '9_Poor_Bridging', 5: '10_Overhang_Sag', 6: '11_Shifted_Layer', 7: '12_Warping', 8: '13_Delamination', 9: '14_Unclear', 10: '15_Not_Printing'}
 obb: None
 orig_img: array([[[ 28,  24,  23],
         [ 28,  24,  23],
         [ 28,  24,  23],
         ...,
         [ 55,  53,  52],
         [ 55,  53,  52],
         [ 55,  53,  52]],
 
        [[ 29,  25,  24],
         [ 28,  24,  23],
         [ 28,  24,  23],
         ...,
         [ 54,  52,  51],
         [ 54,  52,  51],
         [ 54,  52,  51]],
 
        [[ 29,  25,  24],
         [ 29,  25,  24],
         [ 28,  24,  23],
         ...,
         [ 54,  52,  51],
         [ 54,  52,  51],
         [ 53,  51,  50]],
 
        ...,
 
        [[199, 197, 187],
         [204, 202, 192],
         [201, 199, 189],
 

In [13]:
model.val(data="data/data.yaml",
          split="test",
          device=0,
          name="test_metrics"
)

print("✅ Test evaluation complete.")

Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 1100.4544.1 MB/s, size: 301.7 KB)
val: Scanning C:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\test\labels.cache... 200 images, 8 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 6.4it/s 2.0s0.1s
                   all        200        473      0.679      0.514      0.533      0.254
        4_Foreign_Body         21         61      0.729      0.796       0.76       0.32
           6_Stringing         29         64      0.733      0.469        0.5      0.262
           7_Spaghetti         30         51      0.598      0.468      0.481      0.242
       9_Poor_Bridging         31         49       0.85      0.776      0.833      0.452
       10_Overhang_Sag         43         85      0